In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing, model_selection
import string
from sklearn.feature_extraction.text import  CountVectorizer
from scipy.stats import boxcox
from scipy import sparse


In [16]:
train = pd.read_json('train_dummies.json')
test = pd.read_json('test_dummies.json')

y_map = {'low': 2, 'medium': 1, 'high': 0}
train['interest_level'] = train['interest_level'].apply(lambda x: y_map[x])
y_train = train.interest_level.values

x_train = train.drop(['interest_level'], axis = 1)
x_test = test


In [18]:
x_train = sparse.csr_matrix(x_train)
x_test = sparse.csr_matrix(x_test)

In [2]:

SEED = 777
NFOLDS = 5

params = {
    'eta':.01,
    'colsample_bytree':.8,
    'subsample':.8,
    'seed':0,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':0
}

dtrain = xgb.DMatrix(data=x_train, label=y_train)
dtest = xgb.DMatrix(data=x_test)


In [ ]:
bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=25)
#slow because of cv

In [ ]:
best_rounds = np.argmin(bst['test-mlogloss-mean'])#dont run if you dont run cv

In [ ]:
bst = xgb.train(params, dtrain, best_rounds)

In [ ]:
bst.best_iteration

In [76]:
preds = bst.predict(dtest)
preds = pd.DataFrame(preds)
cols = ['high', 'medium', 'low']
preds.columns = cols
preds['listing_id'] = test.listing_id.values



In [ ]:
preds

In [77]:
feats = []
a = bst.get_fscore()
for i in a.keys():
    feats.append([i, a[i]])

In [81]:
preds.to_csv('xgboost.csv', index=None)

In [52]:
x = pd.DataFrame(feats).sort(columns = 1, ascending = False)

/Users/royh21k/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [67]:
x_train2 = x_train[x[x[1]>50][0]]
x_test2 = x_test[x[x[1]>50][0]]
dtest2 = xgb.DMatrix(data=x_test2)
dtrain2 = xgb.DMatrix(data=x_train2, label=y_train)

In [68]:
bst = xgb.train(params, dtrain2, 3500)

In [70]:
preds = bst.predict(dtest2)
preds = pd.DataFrame(preds)
cols = ['high', 'medium', 'low']
preds.columns = cols
preds['listing_id'] = test.listing_id.values
preds.to_csv('xgboost3500treesondummies2reducedcolumns.csv', index=None)

In [72]:
preds

,high,medium,low,listing_id
0,0.043855,0.305511,0.650634,7142618
1,0.070668,0.149924,0.779408,7210040
2,0.036546,0.267875,0.695579,6879707
3,0.396241,0.392448,0.211311,6819133
4,0.023856,0.290096,0.686047,6908212
5,0.036703,0.211997,0.751300,7161974
6,0.109739,0.472971,0.417290,6829365
7,0.040393,0.267232,0.692375,6841710
8,0.189794,0.411070,0.399137,6945954
9,0.010387,0.238625,0.750988,7171409


In [71]:
feats = []
a = bst.get_fscore()
for i in a.keys():
    feats.append([i, a[i]])
x = pd.DataFrame(feats).sort(columns = 1, ascending = False)
x

/Users/royh21k/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,0,1
265,price,23765
127,created,21431
69,bedprice_ratio,21057
35,num_words_desc,17484
134,photo_bri_max,16235
121,photo_bri_min,15652
32,bathprice_ratio,15626
101,photo_bri_mean,14483
9,created_day,12703
28,created_hour,11963
